In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision
from torchvision.models import list_models, get_model
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import time
import copy
import os
import zipfile
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import pickle

In [2]:
with open('preds_formatted.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    preds, losses, accuracies, model_names = pickle.load(f)

In [90]:
preds_shape = (len(preds), len(preds[0]), len(preds[0][0]), len(preds[0][0][0]))
preds_shape
# last batch does not have 128, it has 122

(7, 156, 128, 40)

In [2]:
def format_pred(pred):
    pred = np.array(pred)
    allBatches = []
    for batch in pred:
        for item in batch:
            allBatches = allBatches + [*item]
    return np.array(allBatches).reshape(-1, 40)

# new_preds = [format_pred(pred) for pred in preds[:]]
# new_preds.shape

In [3]:
# real_labels = [test_data[idx][1].numpy() for idx in range(len(test_data))]
# real_labels = np.array(real_labels)

NameError: name 'test_data' is not defined

In [22]:
# real_labels.shape

(19962, 40)

In [4]:
with open('preds_flattened.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    new_preds = pickle.load(f)[0]
with open('labels_flattened.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    real_labels = pickle.load(f)[0]

In [36]:
labels = ['5_o_Clock_Shadow','Arched_Eyebrows','Attractive','Bags_Under_Eyes','Bald','Bangs','Big_Lips','Big_Nose','Black_Hair',
'Blond_Hair', 'Blurry','Brown_Hair','Bushy_Eyebrows','Chubby','Double_Chin','Eyeglasses','Goatee','Gray_Hair','Heavy_Makeup',
'High_Cheekbones','Male','Mouth_Slightly_Open','Mustache','Narrow_Eyes','No_Beard','Oval_Face','Pale_Skin','Pointy_Nose',
'Receding_Hairline','Rosy_Cheeks','Sideburns','Smiling','Straight_Hair','Wavy_Hair','Wearing_Earrings','Wearing_Hat',
'Wearing_Lipstick','Wearing_Necklace','Wearing_Necktie','Young']
labesl_n = [i for i in range(len(labels))]

In [5]:
new_preds.shape

(7, 19962, 40)

In [6]:
real_labels.shape

(19962, 40)

In [74]:
real_labels

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 0, 1]], dtype=int64)

In [7]:
new_preds[:, 0].shape

(7, 40)

In [24]:
real_df = pd.DataFrame(real_labels)
total_labels = real_df.sum(axis=1)
total_labels.head()

0    10
1     9
2    11
3    12
4     7
dtype: int64

In [70]:
real_df.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0,1,0,1,0,0,0,0,0,0,...,0,1,0,1,1,1,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,...,0,0,0,1,0,0,1,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,1
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1


In [125]:
new_preds[0].round().astype(int)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 0, 1]])

In [152]:
new_preds_dfs = [pd.DataFrame(preds) for preds in new_preds]
new_preds_dfs = [df.assign(n_labels=lambda x: total_labels) for df in new_preds_dfs]
new_preds_dfs = [df.assign(sorted_idxs=lambda x: df.apply(lambda x: np.argsort(x[:-1].values)[::-1], axis=1)) for df in new_preds_dfs]
new_preds_dfs = [df.assign(sorted_values=lambda x: df.apply(lambda x: np.sort(x[:-2].values)[::-1], axis=1)) for df in new_preds_dfs]
new_preds_dfs = [df.assign(preds=lambda x: df.apply(lambda x: x.loc[range(40)].values, axis=1)) for df in new_preds_dfs]
new_preds_dfs = [df.reset_index() for df in new_preds_dfs]
new_preds_dfs = [df.assign(rounded=lambda x: df.apply(lambda x: new_preds[i][x['index']].round().astype(int), axis=1)) for i, df in enumerate(new_preds_dfs)]
new_preds_dfs = [df.assign(rounded_set=lambda x: df.apply(lambda x: [idx for idx, item in enumerate(x['rounded']) if item == 1], axis=1)) for i, df in enumerate(new_preds_dfs)]
new_preds_dfs = [df.assign(real=lambda x: df.apply(lambda x: real_df.iloc[x['index'], :].values, axis=1)) for df in new_preds_dfs]
new_preds_dfs = [df.drop(columns = range(40)) for df in new_preds_dfs]

In [156]:
import pyrankvote
from pyrankvote import Candidate, Ballot

In [177]:
candidates = [Candidate(str(l)) for l in labesl_n]

In [183]:
def createBallot(ballot_per_image):
    return [Ballot([candidates[item] for item in pref]) for pref in ballot_per_image]

ballots_per_image = [createBallot([model_preds.loc[i, 'sorted_idxs'] for model_preds in new_preds_dfs]) for i in range(len(real_labels))]

In [ ]:
# get predictions of each model per image
preds_per_image = [[model_preds.loc[i, 'preds'] for model_preds in new_preds_dfs] for i in range(len(real_labels))]
preds_per_image_rounded = [[model_preds.loc[i, 'rounded'] for model_preds in new_preds_dfs] for i in range(len(real_labels))]

In [228]:
def stv(ballots, seats=1):
    election_result = pyrankvote.single_transferable_vote(
        candidates, ballots, number_of_seats=seats
    )
    winners = election_result.get_winners()
    return [int(winner.name) for winner in winners]

In [230]:
stv(ballots_per_image[0])

[24]

In [ ]:
results_df = pd.DataFrame(new_preds_dfs[0].real)

In [212]:
results_df['real_seats'] = results_df.apply(lambda x: [idx for idx, item in enumerate(x['real']) if item == 1], axis=1)
results_df['n_labels'] = new_preds_dfs[0].n_labels
results_df['preds'] = preds_per_image
results_df['preds_rounded'] = preds_per_image_rounded
results_df['ballots_stv'] = ballots_per_image
results_df['soft_voting'] = results_df.apply(lambda x: np.mean(x.preds, axis=0).astype(float).round().astype(int), axis=1)
results_df['hard_voting'] = results_df.apply(lambda x: [1 if i == True else 0 for i in np.sum(x.preds_rounded, axis=0)/len(x.preds_rounded)>=0.5], axis=1)

In [ ]:
results_df['stv_variable_seats'] = results_df.apply(lambda x: stv(x.ballots_stv, x.n_labels) , axis=1)

In [ ]:
results_df.head()

In [ ]:
with open('results.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(results_df, f)